In [1]:
import json
import pandas as pd
from pandas.io.json import json_normalize
import ast
import time
from scipy import stats
from sklearn.cluster import AgglomerativeClustering
import numpy as np, matplotlib.pyplot as plt, time
from sklearn.cluster import DBSCAN
from sklearn import metrics
from geopy.distance import great_circle
from shapely.geometry import MultiPoint
%autosave 60
%matplotlib inline

Autosaving every 60 seconds


In [2]:
#Upload data to Data Frame
df = pd.read_csv("C:/Users/alexd/Downloads/photos_nairobi.csv", header=0)

In [3]:
#Get variables in the right type
df.latitude = df.latitude.astype('float') 
df.longitude = df.longitude.astype('float') 

In [4]:
#Drop null rows
df = df.dropna()

In [5]:
#Drop duplicates
df = df.round({'latitude': 5, 'longitude': 5})
cols = ["customer_id","latitude","longitude","agent_name", "branch", "verification"]
df = df[cols].loc[(df[cols].shift() != df[cols]).any(axis=1)]

In [6]:
# define the number of kilometers in one radian
kms_per_radian = 6371.0088

In [7]:
#Function to return the lat/lon of the center of a cluster of photos
def get_centermost_point(cluster):
    centroid = (MultiPoint(cluster).centroid.x, MultiPoint(cluster).centroid.y)
    centermost_point = min(cluster, key=lambda point: great_circle(point, centroid).m)
    return tuple(centermost_point)

In [11]:
def dbscan_reduce(df, epsilon, x='longitude', y='latitude'):
    start_time = time.time()
    # represent points consistently as (lat, lon) and convert to radians to fit using haversine metric
    coords = df.as_matrix(columns=['latitude', 'longitude'])   
    db = DBSCAN(eps=epsilon, min_samples=1, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))
    cluster_labels = db.labels_
    num_clusters = len(set(cluster_labels))
    print('Number of clusters: {:,}'.format(num_clusters))
    
    clusters = pd.Series([coords[cluster_labels==n] for n in range(num_clusters)])
    
    # find the point in each cluster that is closest to its centroid
    centermost_points = clusters.map(get_centermost_point)

    # unzip the list of centermost points (lat, lon) tuples into separate lat and lon lists
    lats, lons = zip(*centermost_points)
    rep_points = pd.DataFrame({x:lons, y:lats})
    rep_points.tail()
    
    # pull row from original data set where lat/lon match the lat/lon of each row of representative points
    rs = rep_points.apply(lambda row: df[(df[y]==row[y]) & (df[x]==row[x])].iloc[0], axis=1)
    
    clusterdf = pd.DataFrame(columns=['points'])
    clusterdf['points'] = pd.Series(clusters)
    clusterdf['points'] = clusterdf.points.str.len()
    
    rs['number_of_points'] = clusterdf['points']
    return rs

In [8]:
Uniqueids = df.customer_id.unique()

#create a data frame dictionary to store your data frames
DataFrameDict = {elem : pd.DataFrame for elem in Uniqueids}

for key in DataFrameDict.keys():
    DataFrameDict[key] = df[:][df.customer_id == key]

In [9]:
# epsilon=0.05km in radians
eps_rad = 0.05 / kms_per_radian

In [12]:
#Run the clustering algorithm and return the point nearest to the centroid for each cluster
#Then append it to our dataframe
df_clustered = pd.DataFrame()
for key in DataFrameDict:
    df_clustered = df_clustered.append(dbscan_reduce(DataFrameDict[key], epsilon=eps_rad))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
C:\Pr

Number of clusters: 1
Number of clusters: 3
Number of clusters: 1
Number of clusters: 3
Number of clusters: 6
Number of clusters: 1
Number of clusters: 1
Number of clusters: 1
Number of clusters: 6
Number of clusters: 12
Number of clusters: 7
Number of clusters: 1


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
C:\Pr

Number of clusters: 5
Number of clusters: 10
Number of clusters: 1
Number of clusters: 1
Number of clusters: 5
Number of clusters: 1
Number of clusters: 2
Number of clusters: 2
Number of clusters: 3
Number of clusters: 1
Number of clusters: 6
Number of clusters: 9
Number of clusters: 2
Number of clusters: 1


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
C:\Pr

...

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
C:\Pr

Number of clusters: 1
Number of clusters: 1
Number of clusters: 2
Number of clusters: 1
Number of clusters: 1
Number of clusters: 2
Number of clusters: 2
Number of clusters: 8
Number of clusters: 3
Number of clusters: 7


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
C:\Pr

Number of clusters: 2
Number of clusters: 2
Number of clusters: 25
Number of clusters: 1
Number of clusters: 3


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.


In [13]:
#Get the number of customers with an approved photo or at least 3 photos in the same location
total_customers = df_clustered['customer_id'].nunique()
df_identifiable = df_clustered[(df_clustered.number_of_points >= 3) | (df_clustered.verification == True)]
identifiable_customers = df_identifiable['customer_id'].nunique()
print('There are ' + str(total_customers) + ' customers in the dataset. ' + str(identifiable_customers) +', representing ' + str(100*round(identifiable_customers/total_customers,3)) + '% of the total, have an inferrable location.')

There are 4662 customers in the dataset. 1450, representing 31.1% of the total, have an inferrable location.


### Counting customers by branch

In [14]:
branch_total = df_clustered.groupby('branch')['customer_id'].nunique().reset_index()
branch_total = branch_total.rename(columns={"customer_id": "Total"})
branch_identifiable = df_identifiable.groupby('branch')['customer_id'].nunique().reset_index()
branch_identifiable = branch_identifiable.rename(columns={"customer_id": "Identified"})
branch_summary = pd.merge(branch_identifiable, branch_total, how='right', on='branch')
branch_summary['% Identified'] = round(100*branch_summary.Identified/branch_summary.Total,2)
branch_summary

,branch,Identified,Total,% Identified
0,Nairobi East,646,1983,32.58
1,Nairobi West,804,2679,30.01


## Evaluating Photo Effectiveness

In [15]:
#Get the points only for customers that have an inferrable location
df_inferrable = df_clustered[df_clustered.set_index(['customer_id', 'branch']).index.isin(df_identifiable.set_index(['customer_id', 'branch']).index)]

In [16]:
#For each customer, get the number of photos at the inferred point and the number of photos overall
df_inferrable = df_inferrable.groupby(['customer_id', 'branch'])['number_of_points'].agg(
                                  {'Max': np.max, 'Sum': np.sum}
)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  This is separate from the ipykernel package so we can avoid doing imports until


In [17]:
#Calculate the metrics
correct_inf = df_inferrable['Max'].sum()/df_inferrable['Sum'].sum()
total_photos = df['latitude'].count()
pct_correct_photos = df_inferrable['Max'].sum()/total_photos
print('Among the ' + str(identifiable_customers) + ' identifiable customers, ' + str(100*round(correct_inf,4)) + '% of photos are in the correct location.')
print('Among the ' + str(total_photos) + ' photos, an estimated ' + str(100*round(pct_correct_photos,3)) + '% were taken at the correct location.')

Among the 1450 identifiable customers, 55.76% of photos are in the correct location.
Among the 24540 photos, an estimated 38.3% were taken at the correct location.


### Evaluating by Branch

In [18]:
correct_branch = df_inferrable.groupby('branch')['Max'].sum()/df_inferrable.groupby('branch')['Sum'].sum()
correct_branch = correct_branch.reset_index(name='% Closed at Shop')
correct_branch

,branch,% Closed at Shop
0,Nairobi East,0.572991
1,Nairobi West,0.547015


In [23]:
#Drop values of the Nairobi warehouses and Nairobi Dam
df_clustered = df_clustered[(df_clustered.latitude >= -1.28685) | (df_clustered.latitude <= -1.2895) | (df_clustered.longitude <= 36.7555) | (df_clustered.longitude >= 36.76)]
df_clustered = df_clustered[(df_clustered.latitude >= -1.3175) | (df_clustered.latitude <= -1.3272) | (df_clustered.longitude <= 36.898) | (df_clustered.longitude >= 36.911)]
df_clustered = df_clustered[(df_clustered.latitude >= -1.31879) | (df_clustered.latitude <= -1.31975) | (df_clustered.longitude <= 36.7967) | (df_clustered.longitude >= 36.8035)]                                                                      

In [26]:
idx = df_clustered.groupby(['customer_id'])['number_of_points'].transform(max) == df_clustered['number_of_points']

In [27]:
df_clustered2 = df_clustered[idx]

In [28]:
df_clustered2

,customer_id,latitude,longitude,agent_name,branch,verification,number_of_points
0,00220f81-7b1c-e159-29f8-8238068fd3d7,-1.27968,36.75674,Angelo Laban,Nairobi West,False,2
0,00225ec7-5583-40a4-8d87-54e56dd151c5,-1.26622,36.71966,John Njoroge,Nairobi West,False,3
0,002e31b3-d61d-0701-27a5-819dfe862a6c,-1.30472,36.76988,Amos Kihuha,Nairobi West,False,1
0,00311c52-8f80-510a-de60-720f9cee5dc3,-1.29255,36.86711,Wilson Limbe,Nairobi East,False,1
1,00311c52-8f80-510a-de60-720f9cee5dc3,-1.29593,36.86984,Wilson Limbe,Nairobi East,False,1
2,00311c52-8f80-510a-de60-720f9cee5dc3,-1.29806,36.84321,Neville Pele,Nairobi East,False,1
0,0036f4df-a7f9-652e-d494-c3ea151e53ab,-1.26270,36.70163,Justus Onchiri,Nairobi West,False,9
0,003c31a7-5784-4605-8575-d29d46904f11,-1.23280,36.69764,Patrick Maithia,Nairobi West,False,3
0,00412482-c3d0-6e2c-ca4f-9d5f2e6ccb6c,-1.32686,36.89228,Mathew Kwena,Nairobi East,False,1
0,0051f7a7-1b8a-7b45-7ca3-02d063069412,-1.23567,36.89312,Titus Muema,Nairobi East,False,1


In [36]:
import folium
from folium import plugins
#This creates a map with the customers/assignments. I created the color assignments using trial and error.
locations = df_clustered2[['latitude', 'longitude']]
locationlist = locations.values.tolist()
map = folium.Map(location=[df_clustered2['latitude'].mean(), df_clustered2['longitude'].mean()], zoom_start=11)
# plot heatmap
map.add_children(plugins.HeatMap(locationlist, radius=15))
map

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Method `add_children` is deprecated. Please use `add_child` instead.
  
